In [1]:
import numpy as np

import sys
sys.path.append('.')

from sfepy.base.base import output, IndexedStruct, Struct
from sfepy.discrete import (FieldVariable, Material, Integral, Integrals,
                            Equation, Equations, Problem)
from sfepy.discrete.fem import Mesh, FEDomain, Field
from sfepy.discrete.fem.geometry_element import geometry_data
from sfepy.terms import Term
from sfepy.discrete.conditions import Conditions, EssentialBC
from sfepy.solvers.ls import ScipyDirect
from sfepy.solvers.nls import Newton
from sfepy.mechanics.matcoefs import stiffness_from_youngpoisson
from sfepy import data_dir


#from sfepy.examples.linear_elasticity.its2D_2 import stress_strain
#from sfepy.solvers.auto_fallback import AutoDirect
from sfepy.examples.linear_elasticity.its2D_3 import nodal_stress


In [4]:
mesh = Mesh.from_file('triangle2d.mesh')
domain = FEDomain('domain', mesh)

omega = domain.create_region('Omega', 'all')
top = domain.create_region('Top', 'vertex 3', 'vertex')
bottom_left = domain.create_region('Bottom_Left', 'vertex 1', 'vertex')                                        
bottom_right = domain.create_region('Bottom_Right', 'vertex 2', 'vertex')

field = Field.from_args('fu', np.float64, 'vector', omega,
                        approx_order=2)

u = FieldVariable('u', 'unknown', field)
v = FieldVariable('v', 'test', field, primary_var_name='u')

# define elastic properties:
# Aluminum properties in SI units
E = 7.0e10  # Young's Modulus in Pa (70 GPa for Aluminum)
nu = 0.33  # Poisson's ratio for Aluminum
D = stiffness_from_youngpoisson(2, young=E, poisson=nu)

aluminum = Material('Aluminum', D=D)

#set up loading:
# Define forces in Newtons
fy = 6e3  # 1 kn is 1e3
force_ratio = np.tan(np.radians(10))  # Work with an angle of 10 degrees.
fx = fy * force_ratio  # Horizontal force
load = Material('Load', values={'.val' : [fx, fy]})

integral = Integral('i', order=2)
integral0 = Integral('i', order=0)

t1 = Term.new('dw_lin_elastic(Aluminum.D, v, u)',
                integral, omega, Aluminum=aluminum, v=v, u=u)
t2 = Term.new('dw_point_load(Load.val, v)',
                integral0, top, Load=load, v=v)
eq = Equation('balance', t1 - t2)
eqs = Equations([eq])

ls = ScipyDirect({})

nls_status = IndexedStruct()
nls = Newton({}, lin_solver=ls, status=nls_status)

pb = Problem('elasticity', equations=eqs)

# Apply boundary conditions to the two bottom vertices
fix_bottom_left = EssentialBC('fix_bottom_left', bottom_left, {'u.all': 0.0})  # Fix both x and y displacements
fix_bottom_right = EssentialBC('fix_bottom_right', bottom_right, {'u.all': 0.0})  # Fix both x and y displacements

# Add the boundary conditions to the problem setup
pb.set_bcs(ebcs=Conditions([fix_bottom_left, fix_bottom_right]))

pb.set_solver(nls)

# Solve the problem.
variables = pb.solve()
output(nls_status)

# Postprocess the solution.
out = variables.create_output()

ev = pb.evaluate
strain = ev('ev_cauchy_strain.2.Omega(u)', mode='el_avg')
stress = ev('ev_cauchy_stress.2.Omega(Aluminum.D, u)', mode='el_avg',
                copy_materials=False)

out['cauchy_strain'] = Struct(name='output_data', mode='cell',
                                  data=strain, dofs=None)
out['cauchy_stress'] = Struct(name='output_data', mode='cell',
                                  data=stress, dofs=None)

pb.save_state('triangle_load.vtk', out=out)

## Calculate nodal stresses.

# Setup
gdata = geometry_data['2_3']
nc = len(gdata.coors)

integral_vn = Integral('ivn', coors=gdata.coors,
                        weights=[gdata.volume / nc] * nc)

# Update problem time
pb.time_update()

# Evaluate stress at quadrature points
stress = pb.evaluate('ev_cauchy_stress.ivn.Omega(Aluminum.D, u)', mode='qp', 
                        integrals=integral_vn, copy_materials=False)

# Create stress field variable from quadrature points
sfield = Field.from_args('stress', np.float64, (3,), 
                            pb.domain.regions['Surface'])
svar = FieldVariable('sigma', 'parameter', sfield, 
                        primary_var_name='(set-to-None)')
svar.set_from_qp(stress, integrals['ivn'])


sfepy: reading mesh (triangle2d.mesh)...
sfepy:   number of vertices: 279
sfepy:   number of cells:
sfepy:     2_3: 487
sfepy: ...done in 0.00 s
sfepy: updating variables...
sfepy: ...done
sfepy: setting up dof connectivities...
sfepy: ...done in 0.00 s
sfepy: matrix shape: (2084, 2084)
sfepy: assembling matrix graph...
sfepy: ...done in 0.00 s
sfepy: matrix structural nonzeros: 45824 (1.06e+00% fill)
sfepy: updating variables...
sfepy: ...done
sfepy: updating materials...
sfepy:     Load
sfepy:     Aluminum
sfepy: ...done in 0.00 s
sfepy: nls: iter: 0, residual: 6.092560e+03 (rel: 1.000000e+00)
sfepy:   residual:    0.00 [s]
sfepy:     matrix:    0.00 [s]
sfepy:      solve:    0.01 [s]
sfepy: warning: linear system solution precision is lower then the value set in solver options! (err = 1.974852e-09 < 1.000000e-10)
sfepy: nls: iter: 1, residual: 1.998187e-09 (rel: 3.279716e-13)


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.

sfepy: solved in 1 steps in 0.02 seconds
sfepy: IndexedStruct
  condition:
    1
  err:
    1.9981868000187418e-09
  err0:
    6092.5596713144705
  ls_n_iter:
    -1
  n_iter:
    1
  time:
    0.016629916994133964
  time_stats:
    dict with keys: ['residual', 'matrix', 'solve']
sfepy: equation "tmp":
sfepy: ev_cauchy_strain.2.Omega(u)
sfepy: updating materials...
sfepy: ...done in 0.00 s
sfepy: equation "tmp":
sfepy: ev_cauchy_stress.2.Omega(Aluminum.D, u)
sfepy: updating materials...
sfepy:     Aluminum
sfepy: ...done in 0.00 s


Warning: VTK requires 3D vectors, but 2D vectors given. Appending 0 third component to u.